### Número de hipotecas

In [1]:
import requests
import time
import pprint
import pickle
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from wordcloud import WordCloud
import xml.etree.ElementTree as ET
import re
from unicodedata import normalize

In [2]:
def troceador_calles_api(street):
    words_street = []
    words_in_street = street.split(' ')
    for word in words_in_street:
        words = word.split(',')
        for w in words:
            if '/' in w:
                words.extend(w.split('/'))
        for w in words:
            if len(w) > 2 and w.upper() != 'DEL':
                words_street.append(w)
    return words_street
          
def comparador(streets_api, street_to_find):
    words_street_to_find = troceador_calles_api(street_to_find)
    streets = []
    for street in streets_api:
        words_street_api = troceador_calles_api(street)
        coincidences = 0
        for w1 in words_street_api:
            w1 = normalizador(w1)
            len_w1 = len(w1)
            coincidences_words = 0
            for w2 in words_street_to_find:
                w2 = normalizador(w2)
                if len_w1-1 < len(w2) < len_w1+1:
                    if w2 in w1:
                        coincidences_words += 1
            coincidences += coincidences_words
        if coincidences == len(words_street_to_find) and coincidences != 0:
            streets.append(street)
        elif coincidences >= max(int((len(words_street_api)+1)/2), int((len(words_street_to_find)+1)/2)):
            streets.append(street)
    
    if len(streets)==1:
        return streets[0]
    else:
        if len(set(streets))==1:
            return streets[0]
        else:
            for street in streets:
                if normalizador(street) in normalizador(street_to_find):
                    return street
            return False
        
def normalizador(string):
    string = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( "NFD", str(string)), 0, re.I
    )
    string = string.lower()
    return normalize( 'NFC', string)

def parseo_fecha_cuatrimeste(row):
    if row[1] == 'Primero':
        date = '01/01/'
    elif row[1] == 'Segundo':
        date = '01/04/'
    elif row[1] == 'Tercero':
        date = '01/07/'
    elif row[1] == 'Cuarto':
        date = '01/10/'
    return date+str(row[0])

In [3]:
#AL EJECUTAR ESTE SCRIPT ACTUALIZAMOS LAS VIVIENDAS QUE TENEMOS DISPONIBLES PARA EL MODELO
#lA SIGUIENTE TABLA ES LA TABLA FINAL QUE PROVIENE DEL PROYECTO Proyecto_datos_finales de SAS
data_tablin = pd.read_csv(r'C:\Users\Marta\Scripts - Presentación\DATOS_FINAL.csv', 
                   sep=';', engine="python", encoding="latin1")

In [4]:
#DOCUMENTO QUE HEMOS DESCARGADO DEL INE, VER EL LINK DE DESCARGA EN LA DOCUMENTACIÓN
data_ine = pd.read_csv(r'C:\Users\Marta\Desktop\Rotación 2 SBD\BackUp_Proyecto\Python Scripts\Modelo evolución precios\Datos\INE\Hipotecas_constituidas2.csv', 
                   sep=';', skiprows=7, engine="python", decimal='.', encoding="latin1")

data_ine.drop(data_ine.tail(3).index,inplace=True)
data_ine=data_ine.fillna("nan")
data_ine= data_ine.iloc[:,:-1]
data_ine= data_ine[1:]

In [5]:
province_pc = list(data_ine.iloc[:, 0].apply(lambda s: s[:2]))
province_name = list(data_ine.iloc[:, 0].apply(lambda s: s[3:]))


In [6]:
for num_col in range(1, len(data_ine.columns)):
    
    date_year = [data_ine.columns[num_col][:4]]*len(data_ine)
    date_month = [data_ine.columns[num_col][5:7]]*len(data_ine)
    value = list(data_ine.iloc[:, num_col])
    
    df_month = pd.DataFrame({'Código provincia':province_pc, 'Provincia':province_name,
                             'Año':date_year, 'Código mes':date_month, 'Hipotecas':value })
    
    if num_col == 1:
        df_ine = df_month
    else:
        df_ine = df_ine.append(df_month)

In [7]:
df_ine['Hipotecas']=df_ine['Hipotecas'].astype('int')

In [8]:
dict_month = {'01':'Primero', '02':'Primero', '03':'Primero', '04':'Segundo', '05':'Segundo', '06':'Segundo', '07':'Tercero', '08':'Tercero', '09':'Tercero', '10': 'Cuarto', '11':'Cuarto', '12':'Cuarto'} 

df_ine['Cuatrimestre'] = df_ine['Código mes'].map(dict_month)


df_ine['Fecha'] = df_ine.loc[:,['Año', 'Cuatrimestre']].apply(parseo_fecha_cuatrimeste,  axis=1)

col_to_del = ['Año', 'Código mes', 'Cuatrimestre']
for col in col_to_del:
    del df_ine[col]

In [9]:
df_ine['Código provincia']= df_ine['Código provincia'].replace('01', '1')
df_ine['Código provincia']=df_ine['Código provincia'].replace('02', '2')
df_ine['Código provincia']=df_ine['Código provincia'].replace('03', '3')
df_ine['Código provincia']=df_ine['Código provincia'].replace('04', '4')
df_ine['Código provincia']=df_ine['Código provincia'].replace('05', '5')
df_ine['Código provincia']=df_ine['Código provincia'].replace('06', '6')
df_ine['Código provincia']=df_ine['Código provincia'].replace('07', '7')
df_ine['Código provincia']=df_ine['Código provincia'].replace('08', '8')
df_ine['Código provincia']=df_ine['Código provincia'].replace('09', '9')
df_ine.columns= ['CODPROVINC', 'Provincia', 'Hipotecas', 'Fecha']

In [10]:
data_tablin = data_tablin[data_tablin['FCHTASAC'].notnull()]
data_tablin.loc[:,'DIA'] = data_tablin['FCHTASAC'].apply(lambda s: s[:2])
data_tablin.loc[:,'MES'] = data_tablin['FCHTASAC'].apply(lambda s: s[2:5])
data_tablin.loc[:,'AÑO'] = data_tablin['FCHTASAC'].apply(lambda s: int(s[5:9]))
dict_cuatri = {'NOV':'Cuarto', 'SEP':'Tercero', 'OCT':'Cuarto', 'DEC':'Cuarto', 'MAR':'Primero', 'JAN':'Primero', 'MAY':'Segundo', 'FEB':'Primero', 'APR':'Segundo', 'JUN':'Segundo', 'JUL':'Tercero', 'AUG':'Tercero'}
data_tablin['CUATRIMESTRE'] = data_tablin['MES'].map(dict_cuatri)
data_tablin = data_tablin[(data_tablin['AÑO']>1994)&(data_tablin['AÑO']<2019)]

data_tablin['Fecha'] = data_tablin.loc[:,['AÑO', 'CUATRIMESTRE']].apply(parseo_fecha_cuatrimeste,  axis=1)

columns_to_del = ['DIA','CUATRIMESTRE','FCHTASAC']
for col in columns_to_del:
    del data_tablin[col]

In [11]:
data_tablin['CODPROVINC'] = pd.to_numeric(data_tablin['CODPROVINC'])
df_ine['CODPROVINC'] = pd.to_numeric(df_ine['CODPROVINC'])

In [12]:
df_ine['Fecha']= pd.to_datetime(df_ine['Fecha'], format='%d/%m/%Y')
data_tablin['Fecha']= pd.to_datetime(data_tablin['Fecha'], format='%d/%m/%Y')

In [13]:
df_ine=df_ine.groupby(['CODPROVINC', 'Fecha'])['Hipotecas'].mean().round(2).reset_index()

In [14]:
data_tablin= data_tablin.merge(df_ine, left_on=['Fecha', 'CODPROVINC'],
                                    right_on=['Fecha', 'CODPROVINC'], how='left')

### Tasa de desempleo 

In [15]:
data_desempleo = pd.read_csv(r'C:\Users\Marta\Desktop\Rotación 2 SBD\BackUp_Proyecto\Python Scripts\Modelo evolución precios\Datos\INE\Desempleo\desempleo2.csv', 
                   sep=',', engine="python", decimal=',', encoding="latin1")

In [16]:
data_desempleo.index = range(len(data_desempleo))

In [17]:
data_desempleo['Prov']=data_desempleo['Prov'].apply(lambda s: s[:6])

In [18]:
prov_tablin = list(data_tablin.CODPROVINC.unique())
prov_fomento = list(data_desempleo.Prov.unique())

In [19]:
provinces = data_desempleo.Prov
dict_quarter = {'T1':'Primero', 'T2':'Segundo', 'T3':'Tercero', 'T4':'Cuarto'}


In [20]:
for col in data_desempleo.columns[1:]:
    
    year = [col[:4]]*len(data_desempleo)
    quarter = [dict_quarter[col[4:]]]*len(data_desempleo)
    disemploy = data_desempleo.loc[:,col]
    
    if col == '2019T2':
        data_disemploy = pd.DataFrame({'Provincia':provinces, 'Año':year, 'Cuatrimestre':quarter, 'Tasa desempleo':disemploy})
    else:
        data_disemploy = data_disemploy.append(pd.DataFrame({'Provincia':provinces, 'Año':year, 'Cuatrimestre':quarter, 'Tasa desempleo':disemploy}))

In [21]:
data_disemploy['Fecha'] = data_disemploy.loc[:,['Año', 'Cuatrimestre']].apply(parseo_fecha_cuatrimeste,  axis=1)

In [22]:
col_to_del = ['Año', 'Cuatrimestre']
for col in col_to_del:
    del data_disemploy[col]

In [23]:
data_disemploy

,Provincia,Tasa desempleo,Fecha
0,02,17.70,01/04/2019
1,03,14.71,01/04/2019
2,04,16.12,01/04/2019
3,01,8.04,01/04/2019
4,33,14.17,01/04/2019
...,...,...,...
47,47,14.76,01/01/2002
48,49,10.13,01/01/2002
49,50,5.52,01/01/2002
50,51,7.01,01/01/2002


In [24]:
data_disemploy['Provincia']=data_disemploy['Provincia'].replace('01', '1')
data_disemploy['Provincia']=data_disemploy['Provincia'].replace('02', '2')
data_disemploy['Provincia']=data_disemploy['Provincia'].replace('03', '3')
data_disemploy['Provincia']=data_disemploy['Provincia'].replace('04', '4')
data_disemploy['Provincia']=data_disemploy['Provincia'].replace('05', '5')
data_disemploy['Provincia']=data_disemploy['Provincia'].replace('06', '6')
data_disemploy['Provincia']=data_disemploy['Provincia'].replace('07', '7')
data_disemploy['Provincia']=data_disemploy['Provincia'].replace('08', '8')
data_disemploy['Provincia']=data_disemploy['Provincia'].replace('09', '9')

In [25]:
data_disemploy['Provincia'] = pd.to_numeric(data_disemploy['Provincia'])
data_tablin['CODPROVINC'] = pd.to_numeric(data_tablin['CODPROVINC'])
data_disemploy['Fecha']= pd.to_datetime(data_disemploy['Fecha'], format='%d/%m/%Y')

In [26]:
data_tablin= data_tablin.merge(data_disemploy, left_on=['Fecha', 'CODPROVINC'], right_on=['Fecha', 'Provincia'], how='left')

In [27]:
data_tablin.shape

(226068, 32)

In [28]:
data_tablin.to_csv(r'C:\Users\Marta\Scripts - Presentación\tablin_datos_externos.csv', 
                   sep=';', encoding="latin1", decimal=',', index=False)